<a href="https://colab.research.google.com/github/nanmanchu/APTprice/blob/main/6_HyperParameter_Optimazation_Random_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.preprocessing import StandardScaler # Standardization
from keras.callbacks import EarlyStopping # Early Stopping Callback
import warnings
warnings.filterwarnings('ignore') # To ignore warnings.

import xgboost as xgb
from sklearn.metrics import mean_squared_error

from sklearn.metrics import explained_variance_score as evs # evaluation metric
from sklearn.metrics import r2_score as r2 # evaluation metric

from sklearn.metrics import classification_report, accuracy_score

from sklearn.metrics import confusion_matrix

from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
import time

In [ ]:
import IPython

In [ ]:
from sklearn.model_selection import train_test_split
#from category_encoders import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix



#Random Search를 통해 하이퍼파라미터 찾기

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/apt/isol3.csv')

In [ ]:
data

,BEECgroup,Gseed,area,floor,ConstructionYear,OfficialValue,parking,ParkingSpace,pricePerM2,Households,year,age,districtGroup_Dongbuk,districtGroup_Dongnam,districtGroup_Dosim,districtGroup_Seobuk,districtGroup_Seonam
0,0.0,0.0,97.86,6,2005.0,2.820000e+08,2431.0,1.428,5.109340e+04,1702.0,2008,3.0,0,1,0,0,0
1,0.0,0.0,84.78,3,2004.0,1.880000e+08,2431.0,1.428,5.897617e+04,1702.0,2007,3.0,0,1,0,0,0
2,0.0,0.0,56.97,7,1999.0,9.900000e+07,2431.0,1.428,6.143584e+04,1702.0,2010,11.0,0,0,0,0,1
3,0.0,0.0,100.53,2,2008.0,9.900000e+07,2431.0,1.428,7.858351e+04,1702.0,2008,0.0,0,0,0,1,0
4,0.0,0.0,83.70,2,2008.0,9.900000e+07,2431.0,1.428,7.885305e+04,1702.0,2008,0.0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090436,0.0,0.0,59.52,3,1999.0,5.922941e+08,2280.0,1.324,1.444892e+07,1722.0,2018,19.0,0,1,0,0,0
1090437,0.0,0.0,105.89,3,1979.0,9.753663e+08,0.0,0.000,1.444896e+07,1212.0,2017,38.0,0,1,0,0,0
1090438,0.0,0.0,84.78,2,1994.0,5.937931e+08,0.0,0.000,1.444916e+07,638.0,2018,24.0,0,0,1,0,0
1090439,0.0,0.0,55.02,10,1987.0,7.107222e+08,1879.0,0.606,1.444929e+07,3100.0,2019,32.0,0,0,0,0,1


In [ ]:
df=data.copy()

In [ ]:
#정규화
scaler = StandardScaler()  #StandardScaler

# StandardScaler 로 데이터 셋 변환
scaler.fit(df)
tmp = scaler.transform(df)

# DataFrame으로 변환
df = pd.DataFrame(data=tmp, columns=df.columns)

df.describe()

,BEECgroup,Gseed,area,floor,ConstructionYear,OfficialValue,parking,ParkingSpace,pricePerM2,Households,year,age,districtGroup_Dongbuk,districtGroup_Dongnam,districtGroup_Dosim,districtGroup_Seobuk,districtGroup_Seonam
count,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06,1.090441e+06
mean,4.740250e-13,1.345862e-12,-5.052093e-14,3.502163e-14,9.365277e-15,-2.223376e-14,4.714518e-13,-6.489434e-13,-3.194307e-14,1.403278e-13,1.199852e-12,1.226460e-13,1.965644e-12,1.724427e-12,-6.911843e-13,-1.481229e-13,7.388946e-13
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-9.340270e-02,-8.730428e-02,-1.924543e+00,-1.520973e+00,-4.361169e+00,-1.699562e+00,-6.595306e-01,-1.199987e+00,-2.258468e+00,-1.019549e+00,-1.637399e+00,-1.704951e+00,-7.660392e-01,-4.911322e-01,-1.872959e-01,-3.325905e-01,-6.583461e-01
25%,-9.340270e-02,-8.730428e-02,-7.030678e-01,-8.204177e-01,-7.119092e-01,-6.775269e-01,-6.595306e-01,-1.199987e+00,-7.276760e-01,-6.734591e-01,-9.701851e-01,-8.143276e-01,-7.660392e-01,-4.911322e-01,-1.872959e-01,-3.325905e-01,-6.583461e-01
50%,-9.340270e-02,-8.730428e-02,2.439618e-01,-1.198620e-01,1.121173e-01,-2.221151e-01,-3.243371e-01,3.494205e-01,-2.641052e-01,-3.164401e-01,1.418382e-01,-1.463599e-01,-7.660392e-01,-4.911322e-01,-1.872959e-01,-3.325905e-01,-6.583461e-01
75%,-9.340270e-02,-8.730428e-02,3.801107e-01,5.806936e-01,7.007076e-01,4.213558e-01,1.978674e-01,6.318249e-01,5.212724e-01,3.839364e-01,8.090522e-01,6.329357e-01,1.305416e+00,-4.911322e-01,-1.872959e-01,-3.325905e-01,1.518958e+00
max,1.814379e+01,2.296917e+01,3.467728e+00,1.056361e+01,2.701915e+00,1.370159e+01,7.079239e+00,9.967962e+00,3.091611e+00,5.231926e+00,1.698671e+00,4.974726e+00,1.305416e+00,2.036111e+00,5.339145e+00,3.006700e+00,1.518958e+00


In [ ]:
#DMatrix형태로 변환

y = df['pricePerM2']
X = df.drop(['pricePerM2'], axis=1)

data_dmatrix = xgb.DMatrix(data=X,label=y)

#train, test 셋 분리

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=df['BEECgroup'] )

In [ ]:
# model tuning

%%time

params = {    
    'max_depth': [0,5,10],
    'eta': [0.05, 0.08, 0.1], 
    'n_estimators':[500, 1000, 1500], 
    'min_child_weight' : [0, 0.1, 1], 
    'gamma':[0, 0.3, 1],  
    'booster': ['gbtree', 'dart'],
    'subsample':[0, 0.5, 0.8, 1]
}

XGBRegressor = XGBRegressor(nthread=-1, 
                            objective='reg:squarederror')

# run randomized search
random_search = RandomizedSearchCV(XGBRegressor, 
                                   param_distributions=params, 
                                   cv=5, 
                                   scoring='neg_mean_squared_error')

random_search.fit(X_train, y_train)

print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter_search))

RandomizedSearchCV took 4973.36 seconds for 10 candidates parameter settings.
CPU times: user 2d 6h 24min 15s, sys: 18.9 s, total: 2d 6h 24min 34s
Wall time: 1h 22min 53s


In [ ]:
# Get predictions
y_pred = best_regressor.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred)) #np.sqrt 제곱근 계산
print("RMSE: %f" % (rmse))

print('R-Squared of XGBoost model is {}'.format(r2(y_test, y_pred)))

RMSE: 0.233950
R-Squared of XGBoost model is 0.9452223994381984


In [ ]:
best_regressor = random_search.best_estimator_

In [ ]:
best_regressor

XGBRegressor(eta=0.1, gamma=0.3, max_depth=10, min_child_weight=0,
             n_estimators=500, nthread=-1, objective='reg:squarederror', seed=0,
             subsample=0.5)

In [ ]:
#모델 순위 확인
#https://kimmaadata.tistory.com/40
# rank_test_score: 테스트 순위
# mean_score_time: 예측에 걸리는 시간

pd.DataFrame(random_search.cv_results_).sort_values(by='rank_test_score').T

,2,3,9,7,0,1,4,5,6,8
mean_fit_time,126.292043,132.013993,674.309107,32.106709,0.310601,0.306786,0.292318,0.289335,0.287728,0.288216
std_fit_time,15.989614,1.921792,29.778372,0.130356,0.008666,0.005277,0.004467,0.003384,0.001526,0.000968
mean_score_time,0.452623,0.484698,0.308039,0.053492,0.0,0.0,0.0,0.0,0.0,0.0
std_score_time,0.005444,0.065291,0.004858,0.000242,0.0,0.0,0.0,0.0,0.0,0.0
param_subsample,0.5,1,0.5,0.5,0,0,1,0,0.5,0.8
param_seed,0,0,2019,0,0,2019,2019,0,2019,2019
param_objective,reg:squarederror,reg:squarederror,reg:squarederror,reg:squarederror,reg:logistic,reg:logistic,reg:logistic,reg:logistic,reg:logistic,reg:logistic
param_n_estimators,500,1500,1000,1500,1000,500,1000,1500,1000,500
param_min_child_weight,0,0,0.1,0.1,0.1,0,1,0,0,1
param_max_depth,10,5,5,0,10,0,0,0,0,10


In [ ]:
cvres = random_search.cv_results_  
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
  print(np.sqrt(-mean_score), params)


nan {'subsample': 0, 'seed': 0, 'objective': 'reg:logistic', 'n_estimators': 1000, 'min_child_weight': 0.1, 'max_depth': 10, 'gamma': 0, 'eta': 0.05, 'booster': 'dart'}
nan {'subsample': 0, 'seed': 2019, 'objective': 'reg:logistic', 'n_estimators': 500, 'min_child_weight': 0, 'max_depth': 0, 'gamma': 1, 'eta': 0.05, 'booster': 'gbtree'}
0.236387741861681 {'subsample': 0.5, 'seed': 0, 'objective': 'reg:squarederror', 'n_estimators': 500, 'min_child_weight': 0, 'max_depth': 10, 'gamma': 0.3, 'eta': 0.1, 'booster': 'gbtree'}
0.25893600794901694 {'subsample': 1, 'seed': 0, 'objective': 'reg:squarederror', 'n_estimators': 1500, 'min_child_weight': 0, 'max_depth': 5, 'gamma': 0.3, 'eta': 0.1, 'booster': 'gbtree'}
nan {'subsample': 1, 'seed': 2019, 'objective': 'reg:logistic', 'n_estimators': 1000, 'min_child_weight': 1, 'max_depth': 0, 'gamma': 1, 'eta': 0.08, 'booster': 'gbtree'}
nan {'subsample': 0, 'seed': 0, 'objective': 'reg:logistic', 'n_estimators': 1500, 'min_child_weight': 0, 'max_d